In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import json
import os

# 1. JSON 파일에서 데이터를 불러오기
json_file_path = 'audio_features_data.json'  # json 파일의 이름이 실제 파일 이름과 일치해야 합니다

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    audio_features_data = json.load(f)

# values()를 사용하여 데이터만 추출하고 numpy array로 변환
data = np.array(list(audio_features_data.values()))

# 분위기 레이블: 예시로 3개의 레이블을 부여 (각 노래에 실제 레이블을 할당해야 합니다)
# 예시 레이블, 실제 데이터에 맞게 수정하세요
num_samples = len(data)

labels = [0] * 100 + [1] * 69 + [2] * (num_samples - 169)

In [31]:
data

array([[1.20073e+02, 3.63000e-01, 7.47000e-01, ..., 7.30000e-03,
        8.15000e-02, 7.38000e-02],
       [1.24994e+02, 7.57000e-01, 8.95000e-01, ..., 8.16000e-02,
        6.23000e-02, 1.54000e-01],
       [1.19986e+02, 8.01000e-01, 8.86000e-01, ..., 1.30000e-01,
        1.85000e-01, 3.09000e-01],
       ...,
       [8.70930e+01, 5.71000e-01, 5.85000e-01, ..., 6.24000e-01,
        6.10000e-02, 1.25000e-01],
       [8.00740e+01, 1.75000e-01, 4.50000e-01, ..., 6.57000e-01,
        2.95000e-02, 1.39000e-01],
       [2.01801e+02, 3.68000e-01, 6.45000e-01, ..., 4.39000e-01,
        3.00000e-02, 1.49000e-01]])

In [32]:
# 2. PyTorch Dataset 클래스 정의
class MusicDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]

# 데이터 차원 확장 (CNN 입력으로 2D 필요, Batch x Channel x Height x Width)
data = data[:, np.newaxis, np.newaxis, :]  # (batch_size, 1, 1, 7) 형태로 변경

# PyTorch Dataset과 DataLoader 정의는 기존 코드와 동일하게 사용
dataset = MusicDataset(data, labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


# 3. 간단한 CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(1, 1))  # 커널 크기 변경
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(1, 1))
        self.fc1 = nn.Linear(32 * 7, 64)  # 피쳐맵 크기를 일자로 펼치기 (1 x 7의 결과가 32채널이므로 32*7)
        self.fc2 = nn.Linear(64, 3)  # 분위기 레이블 수 (예: 3가지 레이블)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d((1, 1))(x)  # MaxPooling은 (1, 1)으로 설정하여 크기 변경하지 않음
        x = nn.ReLU()(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x


# 4. 모델, 손실 함수 및 옵티마이저 설정
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. 모델 학습
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/100], Loss: 2.9707
Epoch [2/100], Loss: 0.7226
Epoch [3/100], Loss: 1.2376
Epoch [4/100], Loss: 0.8248
Epoch [5/100], Loss: 0.1408
Epoch [6/100], Loss: 0.1930
Epoch [7/100], Loss: 0.4995
Epoch [8/100], Loss: 1.2547
Epoch [9/100], Loss: 0.1523
Epoch [10/100], Loss: 0.2480
Epoch [11/100], Loss: 1.1154
Epoch [12/100], Loss: 0.9222
Epoch [13/100], Loss: 1.6223
Epoch [14/100], Loss: 0.1510
Epoch [15/100], Loss: 0.2264
Epoch [16/100], Loss: 0.5008
Epoch [17/100], Loss: 1.3896
Epoch [18/100], Loss: 0.4421
Epoch [19/100], Loss: 0.3111
Epoch [20/100], Loss: 1.1300
Epoch [21/100], Loss: 2.2384
Epoch [22/100], Loss: 0.5805
Epoch [23/100], Loss: 0.5843
Epoch [24/100], Loss: 0.1054
Epoch [25/100], Loss: 0.1937
Epoch [26/100], Loss: 0.1628
Epoch [27/100], Loss: 2.0857
Epoch [28/100], Loss: 0.3001
Epoch [29/100], Loss: 0.4065
Epoch [30/100], Loss: 1.6345
Epoch [31/100], Loss: 0.7442
Epoch [32/100], Loss: 0.0548
Epoch [33/100], Loss: 0.2776
Epoch [34/100], Loss: 0.3488
Epoch [35/100], Loss: 0

In [33]:
# 모델 저장 경로 설정
model_save_path = "cnn_learning.pth"

# 모델의 state_dict를 저장
torch.save(model.state_dict(), model_save_path)
print(f"모델이 저장되었습니다: {model_save_path}")


모델이 저장되었습니다: cnn_learning.pth


In [34]:
# 모델을 새로 초기화하고 state_dict 불러오기
loaded_model = SimpleCNN()  # 모델 구조는 동일해야 합니다
loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.eval()  # 평가 모드로 변경 (드롭아웃, 배치 정규화 등 비활성화)


SimpleCNN(
  (conv1): Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=224, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)

In [43]:
# 예시 입력 데이터 (학습 시 사용했던 형식과 동일한 형태로 준비해야 함)
sample_data = np.array([[
        116.08,
        0.834,
        0.876,
        0.855,
        0.00857,
        0.166,
        0.148
    ]])  # 예시 데이터
sample_data = sample_data[:, np.newaxis, np.newaxis, :]  # (1, 1, 1, 7) 형태로 변환
sample_data_tensor = torch.tensor(sample_data, dtype=torch.float32)

# 모델로 예측하기
with torch.no_grad():  # 예측 시에는 gradient 계산이 필요하지 않으므로 no_grad() 사용
    prediction = loaded_model(sample_data_tensor)
    predicted_label = torch.argmax(prediction, dim=1).item()

print(f"Predicted Label: {predicted_label}")


Predicted Label: 0
